In [1]:
%pip install -q requests beautifulsoup4 cloudscraper

Note: you may need to restart the kernel to use updated packages.


At this point you need a library to proceed in the format:
```
https://www.livechart.me/users/#UserID/library
```

In [2]:
from bs4 import BeautifulSoup

import cloudscraper

scraper = cloudscraper.create_scraper()  # This simulates a browser and can handle Cloudflare
response = scraper.get('https://www.livechart.me/users/Marrro90123/library').text

soup = BeautifulSoup(response, 'html.parser')
titles = soup.find_all('a', class_='link link-hover font-medium')  # get all titles

for title in titles:
    print(title.get_text().strip())


Chanto Suenai Kyuuketsuki-chan
Gachiakuta
One-Punch Man Season 3
Sawaranaide Kotesashi-kun
Egao no Taenai Shokuba desu.
Isekai Quartet3
3-nen Z-gumi Ginpachi-sensei
Chitose-kun wa Ramune Bin no Naka
Tondemo Skill de Isekai Hourou Meshi 2
Yano-kun no Futsuu no Hibi
Ganso! Bandori-chan
Bukiyou na Senpai.
Shinjiteita Nakama-tachi ni Dungeon Okuchi de Korosarekaketa ga Gift "Mugen Gacha" de Level 9999 no Nakama-tachi wo Te ni Irete Moto Party Member to Sekai ni Fukushuu & "Zamaa!" Shimasu!
Watari-kun no xx ga Houkai Sunzen
Kekkon Yubiwa Monogatari Ⅱ
SPY x FAMILY Season 3
Ranma1/2 (2024) 2nd Season
Tomodachi no Imouto ga Ore ni dake Uzai
Princession Orchestra
Xian Wang de Richang Shenghuo 5
Umamusume: Cinderella Gray Cour 2
UNDEAD UNLUCK Winter-hen
CAT'S❤︎EYE
Kaguya-sama wa Kokurasetai: Otona e no Kaidan
Yuusha-kei ni Shosu: Choubatsu Yuusha 9004-tai Keimu Kiroku
Fate/strange Fake
Kirei ni Shitemoraemasu ka.
Osananajimi to wa LoveCom ni Naranai
Mayonaka Heart Tune
Yuusha Party ni Kawaii Ko 

In [3]:
# write the titles to a text file for manual review (you dont want to end up with duplicates)
# new feature: you can specify "must contain" after "->" in the titles.txt file
import os

with open(os.path.join(os.getcwd(), 'titles.txt'), 'w') as f:
    for title in titles:
        f.write(title.get_text().strip() + '\n')

This is where the fun begins.
when we are done with titles.txt, we can update the rules with the new titles.
Rules generated will go to a specified savepath, which can be edited in this part of the code.
```
"priority": 0,
            "savePath": "C:\\PATH\\TO\\SAVE\\FOLDER",
            "smartFilter": False,
```

The rss link will be added in the update_rules function like this
```
update_rules(rules, titles, 'https://path/to/rss/link') 
```
after thiis function is called, you will have to save the updated rules to the same rules file.


In [4]:
import json

def load_titles(file_path):
    """Load anime titles and must-contain from a text file.

    Format per line:
    Title [optional cleanup like trailing '.'] -> must|contain|string

    If '->' is present, the right side is used as-is for mustContain.
    If absent, mustContain defaults to the cleaned title.
    """
    titles = []
    must_contain = []
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.read().splitlines()

    for line in lines:
        if not line.strip():
            continue
        if "->" in line:
            left, right = line.split("->", 1)
            title = left.strip().rstrip(".")
            must = right.strip()  # keep as-is, e.g. "something|some"
        else:
            title = line.strip().rstrip(".")
            must = title  # default to title if no explicit must-contain
        titles.append(title)
        must_contain.append(must)

    return titles, must_contain

def load_rules(json_path):
    """Load the current rules from a JSON file."""
    with open(json_path, 'r', encoding='utf-8') as file:
        rules = json.load(file)
    return rules

def update_rules(rules, titles, must_contain, link):
    """Update rules with new titles and aligned must-contain strings."""
    for title, must in zip(titles, must_contain):
        rules[f"\"{title}\""] = {
            "addPaused": None,
            "affectedFeeds": [f"{link}"],
            "assignedCategory": "",
            "enabled": True,
            "episodeFilter": "",
            "ignoreDays": 0,
            "lastMatch": "",
            "mustContain": must,  # preserves e.g. "something|some"
            "mustNotContain": "",
            "previouslyMatchedEpisodes": [],
            "priority": 0,
            "savePath": "C:\\ANYMAY",
            "smartFilter": False,
            "torrentContentLayout": None,
            "torrentParams": {
                "category": "",
                "download_limit": -1,
                "download_path": "",
                "inactive_seeding_time_limit": -2,
                "operating_mode": "AutoManaged",
                "ratio_limit": -2,
                "save_path": "C:/ANYMAY",
                "seeding_time_limit": -2,
                "skip_checking": False,
                "tags": [],
                "upload_limit": -1,
                "use_auto_tmm": False
            },
            "useRegex": False
        }

def save_rules(rules, json_path):
    """Save the updated rules to a JSON file."""
    with open(json_path, 'w', encoding='utf-8') as file:
        json.dump(rules, file, indent=4)

# Paths to your files
titles_file = 'titles.txt'
rules_json = 'rules.json'

# Process the update
titles, must_contain = load_titles(titles_file)
rules = load_rules(rules_json)
# remember to add link
# print(titles)
# print(must_contain)
update_rules(rules, titles, must_contain, 'https://www.erai-raws.info/episodes/feed/?res=1080p&type=magnet&subs[]=us&token=586898edfa014dda1817789b8c252d75')
save_rules(rules, rules_json)


now you can start using the new rules in the torrent client of your choice, this script was tested with qBittorrent 4.6.5.10